# Parallel computing with Dask

## Authors & Contributors
### Authors
- Tina Odaka, Ifremer (France), [@tinaok](https://github.com/tinaok)
- Anne Fouilloux, Simula Research Laboratory (Norway), [@annefou](https://github.com/annefou)
- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)

### Contributors
- Guillaume Eynard-Bontemps, CNES (France), [@guillaumeeb](https://github.com/guillaumeeb)


<div class="alert alert-info">
<i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> Overview
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>What is Dask?</li>
        <li>How can I parallelize my data analysis with Dask?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn about Dask</li>
        <li>Learn about Dask Gateway, Dask Client, Scheduler, Workers</li>
        <li>Understand out-of-core and speed-up limitations</li>
    </ul>
</div>

## Context


We will be using [Dask](https://docs.dask.org/) with [Xarray](https://docs.xarray.dev/en/stable/) to parallelize our data analysis. The analysis is very similar to what we have done in previous episodes but this time we will use data on a global coverage that we read from a shared catalog (stored online in the Pangeo EOSC Openstack Object Storage).

### Data

In this episode, we will be using CMIP6 data from intake-esm catalogue

## Setup

This episode uses the following Python packages:

- pooch {cite:ps}`e-pooch-Uieda2020`
- s3fs {cite:ps}`e-s3fs-2016`
- xarray {cite:ps}`e-xarray-hoyer2017` with [`netCDF4`](https://pypi.org/project/h5netcdf/) and [`h5netcdf`](https://pypi.org/project/h5netcdf/) engines
- hvplot {cite:ps}`e-holoviews-rudiger2020`
- dask {cite:ps}`e-dask-2016`
- graphviz {cite:ps}`e-graphviz-Ellson2003`
- numpy {cite:ps}`e-numpy-harris2020`
- pandas {cite:ps}`e-pandas-reback2020`
- geopandas {cite:ps}`e-geopandas-jordahl2020`

Please install these packages if not already available in your Python environment (you might want to take a look at [the Setup page of the tutorial](https://pangeo-data.github.io/clivar-2022/before/setup.html)).
### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

## Parallelize with Dask

We know from previous chapter [chunking_introduction](./chunking_introduction.ipynb) that chunking is key for analyzing large datasets. In this episode, we will learn to parallelize our data analysis using [Dask](https://docs.dask.org/) on our chunked dataset. 

### What is [Dask](https://docs.dask.org/) ?

**Dask** scales the existing Python ecosystem: with very or no changes in your code, you can speed-up computation using Dask or process bigger than memory datasets.

- Dask is a flexible library for parallel computing in Python.
- It is widely used for handling large and complex Earth Science datasets and speed up science.
- Dask is powerful, scalable and flexible. It is the leading platform today for data analytics at scale.
- It scales natively to clusters, cloud, HPC and bridges prototyping up to production.
- The strength of Dask is that is scales and accelerates the existing Python ecosystem e.g. Numpy, Pandas and Scikit-learn with few effort from end-users.

It is interesting to note that at first, [Dask has been created to handle data that is larger than memory, on a single computer](https://coiled.io/blog/history-dask/). It then was extended with Distributed to compute data in parallel over clusters of computers.

#### How does Dask scale and accelerate your data analysis?

[Dask proposes different abstractions to distribute your computation](https://docs.dask.org/en/stable/10-minutes-to-dask.html). In this _Dask Introduction_ section, we will focus on [Dask Array](https://docs.dask.org/en/stable/array.html) which is widely used in pangeo ecosystem as a back end of Xarray.

As shown in the [previous section](./chunking_introduction.ipynb) Dask Array is based on chunks.
Chunks of a Dask Array are well-known Numpy arrays. By transforming our big datasets to Dask Array, making use of chunk, a large array is handled as many smaller Numpy ones and we can compute each of these chunks independently.

![Dask and Numpy](https://examples.dask.org/_images/dask-array-black-text.svg)



<div class="alert alert-info">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Warning</b>
    <br>
    <ul>
        <li>`Xarray` uses Dask Arrays instead of Numpy when chunking is enabled, and thus all Xarray operations are performed through Dask, which enables distributed processing. </li>
    </ul>
</div>


#### How does Xarray with Dask distribute data analysis?

When we use chunks with `Xarray`, the real computation is only done when needed or asked for, usually when invoking `compute()` function. Dask generates a **task graph** describing the computations to be done. When using [Dask Distributed](https://distributed.dask.org/en/stable/) a **Scheduler** distributes these tasks across several **Workers**.

![Xarray with dask](../figures/dask-xarray-explained.png)

#### What is a Dask Distributed ?

A Dask Distributed is made of two main components:

- a Scheduler, responsible for dishandling computations graph and distributing tasks to Workers.
- One or several (up to 1000s) Workers, computing individual tasks and storing results and data into distributed memory (RAM and/or worker's local disk).

A user usually needs distributed __Client__ and __Cluster__ object as shown below to use Dask Distribute.    

![Dask Distributed Cluster](https://user-images.githubusercontent.com/306380/66413985-27111600-e9be-11e9-9995-8f418ff48f8a.png)

#### Where can we deploy Dask distributed cluster?  
[Dask distributed clusters can be deployed on your laptop or on distributed infrastructures ( Cloud, HPC centers or ..).](https://docs.dask.org/en/stable/deploying.html)  Dask distributed `Cluster` object is responsible of deploying and scaling a Dask Cluster on the underlying resources.

![Dask Cluster deployment](https://docs.dask.org/en/stable/_images/dask-cluster-manager.svg)

:::{tip}
A Dask `Cluster` can also be created on a single machine (for instance your laptop) e.g. there is no need to have dedicated computational resources. However, speedup will only be limited to your single machine resources if you do not have dedicated computational resources!
:::

### Dask distributed Client
 
The Dask distributed `Client` is what allows you to interact with Dask distributed Clusters. When using Dask distributed, you always need to create a `Client` object. Once a `Client` has been created, it will be used by default by each call to a Dask API, even if you do not explicitly use it.

No matter the Dask API (e.g. Arrays, Dataframes, Delayed, Futures, etc.) that you use, under the hood, Dask will create a Directed Acyclic Graph (DAG) of tasks by analysing the code. Client will be responsible to submit this DAG to the Scheduler along with the final result you want to compute. The Client will also gather results from the Workers, and aggregates it back in its underlying Python process.

Using `Client()` function with no argument, you will create a local Dask cluster with a number of workers and threads per worker corresponding to the number of cores in the 'local' machine. Here, during the workshop, we are running this notebook in Pangeo EOSC cloud deployment, so the 'local' machine is the jupyterlab you are using at the Cloud, and the number of cores is the number of cores on the cloud computing resources you've been given (not on your laptop).

In [ ]:
from dask.distributed import Client

client = Client()  # create a local dask cluster on the local machine.
client

Inspecting the `Cluster Info` section above gives us information about the created cluster: we have 2 or 4 workers and the same number of threads (e.g. 1 thread per worker). 



<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Go further</b>
    <br>
    <ul>
        <li> You can also create a local cluster with the `LocalCluster` constructor and use `n_workers` 
        and `threads_per_worker` to manually specify the number of processes and threads you want to use. 
        For instance, we could use `n_workers=2` and `threads_per_worker=2`.  </li>
        <li> This is sometimes preferable (in terms of performance), or when you run this tutorial on your PC, 
        you can avoid dask to use all your resources you have on your PC!  </li>
    </ul>
</div>

### Dask Dashboard

Dask comes with a really handy interface: the Dask Dashboard.
We will learn here, how to use that through [dask jupyterlab extension](https://github.com/dask/dask-labextension).  

To use Dask Dashbord through jupyterlab extension at Pangeo EOSC FOSS4G infrastructure,
you will just need too look at the html link you have for your jupyterlab, and Dask dashboard port number, as highlighted in the figure below.


<img src="../figures/dashboardlink.png" width="50%">
<img src="../figures/dasklab.png" width="30%">

Then click the orange icon indicated in the above figure, and type 'your' dashboard link (normally, you just need to replace 'todaka' to 'your username').  






You can click several buttons indicated with blue arrows in above figures, then drag and drop to place them as your convenience.  


<img src="../figures/exampledasklab.png" width="50%">


It's really helpfull to understand your computation and how it is distributed.

## Dask Distributed computations on our dataset

Lets open dataset from catalogue we made before, select a single location over time and visualize the task graph generated by Dask, and observe the Dask Dashboard.
### Read from online cmip6 ARCO dataset

 We will access analysis-ready, cloud optimized (ARCO) introduced in [previous data_discovery](./data_discovery.ipynb) section.

In [ ]:
import xarray as xr

xr.set_options(display_style="html")
import intake
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import hvplot.xarray

%matplotlib inline
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
cat = col.search(
    experiment_id=["historical"]
    # , table_id=['day']
    # source_id=['NorESM2-LM']
    ,
    institution_id=["NIMS-KMA"]
    # , table_id=['Amon']
    ,
    table_id=["3hr"],
    variable_id=["tas"],
    member_id=["r1i1p1f1"],
    grid_label=["gr"],
)
display(cat.df)
dset_dict = cat.to_dataset_dict(zarr_kwargs={"use_cftime": True})
dset = dset_dict[list(dset_dict.keys())[0]]

In [ ]:
dset

## Compute Weigted arctic average
Let's try to also take only the data above 60$^\circ$

In [ ]:
dset = dset.where(dset["lat"] > 60.0, drop=True)  #
# this line limiting time is only for building jupyter book
dset = dset.sel(time=slice("2000-01-01", "2010-01-01"))
dset

By inspecting the variable 'tas' on the representation above, you'll see that data array represent __about 8.2 GiB of data__, so more thant what is available on this notebook server, i.e. even on the local Dask Cluster we created above. But thanks to chunking, we can still analyze it!

Lets plot the first timestep

In [ ]:
projection = ccrs.Mercator(central_longitude=-10)

f, ax = plt.subplots(subplot_kw=dict(projection=projection))

dset["tas"].isel(time=0).plot(
    transform=ccrs.PlateCarree(), cbar_kwargs=dict(shrink=0.7), cmap="coolwarm"
)
ax.coastlines()

## Compute weighted mean

1. Creating weights: for a rectangular grid the cosine of the latitude is proportional to the grid cell area.
2. Compute weighted mean values

In [ ]:
def computeWeightedMean(ds):
    # Compute weights based on the xarray you pass
    weights = np.cos(np.deg2rad(ds.lat))
    weights.name = "weights"
    # Compute weighted mean
    air_weighted = ds.weighted(weights)
    weighted_mean = air_weighted.mean(("lon", "lat"))
    return weighted_mean

In [ ]:
weighted_mean = computeWeightedMean(dset)
weighted_mean

Did you notice something on the Dask Dashboard when running the two previous cells?

We didn't 'compute' anything. We just build a Dask task graph with it's size indicated as count above, but did not ask Dask to return a result.


You can try to plot the dask graph before computation and understand what dask workers will do to compute the value we asked for. 

In [ ]:
weighted_mean.tas.data.visualize()

### Lets compute using dask!

In [ ]:
%%time
weighted_mean = weighted_mean.compute()

Calling compute on our Xarray object triggered the execution on Dask Cluster side. 

You should be able to see how Dask is working on Dask Dashboard. 

In [ ]:
weighted_mean.tas.plot()

### Close client to terminate local dask cluster

The `Client` and associated `LocalCluster` object will be automatically closed when your Python session ends. When using Jupyter notebooks, we recommend to close it explicitely whenever you are done with your local Dask cluster.

In [ ]:
client.close()

## Scaling your Computation using Dask Gateway.

For this workshop, according to the Pangeo EOSC deployment, you will learn how to use Dask Gateway to manage Dask clusters over Kubernetes, allowing to run our data analysis in parallel e.g. distribute tasks across several workers.

Lets set up your Dask Gateway.  
As Dask Gateway is configured by default on this ifnrastructure, you just need to execute the following cells.

In [ ]:
from dask_gateway import Gateway

gateway = Gateway()

In [ ]:
##A line of trick to clean your dask cluster before you start your computation
from dask.distributed import Client

clusters = gateway.list_clusters()
print(clusters)
for cluster in clusters:
    cluster = gateway.connect(cluster.name)
    print(cluster)
    client = Client(cluster)
    client.close()
    cluster.shutdown()

### Create a new Dask cluster with the Dask Gateway

In [ ]:
## Please don't execute this cell, it is needed for building the Jupyter Book
cluster = None

In [ ]:
cluster = gateway.new_cluster(worker_memory=2, worker_cores=1)

cluster.scale(8)
cluster

Lets setup the Dask Dashboard with your new cluster.  
This time, copy and past the link above indicated in Dashboard to the dasklab extension.

### Get a client from the Dask Gateway Cluster

As stated above, creating a Dask `Client` is mandatory in order to perform following Daks computations on your Dask Cluster.

In [ ]:
from distributed import Client

if cluster:
    client = Client(cluster)  # create a dask Gateway cluster
else:
    client = Client()  # create a local dask cluster on the machine.
client

## Repeat above computation with more dask workers.  


In [ ]:
dset

In [ ]:
weighted_mean = computeWeightedMean(dset)
weighted_mean

In [ ]:
%%time
weighted_mean = weighted_mean.compute()

## Comparison with unweighted mean
- We select a time range
- Note how the weighted mean temperature is higher than the unweighted.

In [ ]:
weighted_mean["tas"].sel(time=slice("2000-01-01", "2010-01-01")).plot(label="weighted")
dset["tas"].sel(time=slice("2000-01-01", "2010-01-01")).mean(("lon", "lat")).plot(
    label="unweighted"
)

plt.legend()

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Exercise</b>
    <br>
    <ul>
        <li> It was faster.  Why? </li>          
    </ul>
</div>

## hvplot and dask

In [ ]:
dset.isel(member_id=0).hvplot.quadmesh(  # time=0,
    x="lon",
    y="lat",
    rasterize=True,
    geo=True,
    global_extent=False,
    projection=ccrs.Orthographic(30, 90),
    project=True,
    cmap="coolwarm",
    coastline="50m",
    frame_width=400,
    title=dset.attrs["references"]
    # , title="Near-surface Temperature over Norway (CMIP6 CESM2)"
)

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Go Further</b>
    <br>
    <ul>
        <li>Compare the data size and 'used' data size for each worker in dask dashboard  </li>   
        <li>Lets try to zoom.  What happend with your plot? How was the dask dashboard reacted with zooming?   </li>   
        <li>What is rastersize=True ? (Hint: https://hvplot.holoviz.org/user_guide/Customization.html#datashading-options)   </li>  
        <li>What is 'cluster.scale(4)' ? </li>
        <li>instead of .compute, try using .persist, how's the dashboard?
    </ul>
</div>


In [ ]:
client.close()

In [ ]:
cluster.shutdown()

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Go Further </b>
    <br>
    <ul>
        <li>Dask will try to hold data on the memory, then try to spill that to hard disk of worker.  If you would like to avoid that dask worker use your local disk (It may slow down your computation depending on what kind of hardware you have), you can use following command after importing dask.distributed. </li>          
        dask.config.set({"distributed.worker.memory.spill": 0})
    </ul>
</div>

## Packages citation

```{bibliography}
:style: alpha
:filter: topic % "dask" and topic % "package"
:keyprefix: e-
```